Using Fine Tuning - Transfer Learning technique to understand Keras Functional Model to predict multiple outputs.

In [88]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [89]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [90]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [91]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [92]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [93]:
age = []
gender = []
img_path = []
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [94]:
len(age)

23708

In [95]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [96]:
df.shape

(23708, 3)

In [97]:
df.head()

,age,gender,img
0,28,0,28_0_0_20170117020729301.jpg.chip.jpg
1,74,0,74_0_0_20170120225142161.jpg.chip.jpg
2,16,0,16_0_0_20170110231918993.jpg.chip.jpg
3,49,1,49_1_0_20170117160608302.jpg.chip.jpg
4,27,0,27_0_2_20170119193327146.jpg.chip.jpg


In [98]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [99]:
train_df.shape

(20000, 3)

In [100]:
test_df.shape

(3708, 3)

In [101]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [102]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=(224, 224),
    batch_size=32,  # Set an appropriate batch size
    class_mode='raw',  # Assuming age and gender are not one-hot encoded
    # Add other relevant arguments like shuffle, seed, etc.
)
test_generator = test_datagen.flow_from_dataframe(test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],
    target_size=(224, 224),
    batch_size=32,  # Set an appropriate batch size
    class_mode='raw',  # Assuming age and gender are not one-hot encoded
    # Add other relevant arguments like shuffle, seed, etc.
)

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [102]:
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(224,224),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(224,224),
                                                  class_mode='multi_output')

In [103]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model
import tensorflow as tf

In [104]:
resnet = ResNet50(include_top=False, input_shape=(224,224,3))

In [105]:
resnet = ResNet50(include_top=False, input_shape=(224,224,3))

resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(32, activation='relu')(flatten)
dense2 = Dense(32,activation='relu')(flatten)

dense3 = Dense(32,activation='relu')(dense1)
dense4 = Dense(32,activation='relu')(dense2)

output1 = Dense(32,activation='linear',name='age')(dense3)
output2 = Dense(32,activation='sigmoid',name='gender')(dense4)

In [106]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [108]:
model.compile(optimizer='adam',
              loss={'age': 'mae', 'gender': 'binary_crossentropy'},
              metrics={'age': 'mae', 'gender': 'accuracy'},
              loss_weights={'age':1.0,'gender':99.0},
              run_eagerly=True)

In [109]:
model.fit(train_generator, batch_size=8, epochs=5, validation_data=test_generator)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:GPU:0}} required broadcastable shapes [Op:Sub] name: 